In [19]:
import numpy as np
import pickle
import scipy
import h5py
import scipy.io as sio
from pyriemann.utils.mean import mean_covariance
import sklearn.datasets
import sklearn.decomposition
from scipy.spatial import distance

np.seterr(divide='ignore', invalid='ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

In [24]:
def utri2mat(utri):
    n = int(-1 + np.sqrt(1 + 8 * len(utri))) // 2
    iu1 = np.triu_indices(n+1,1)
    ret = np.empty((n+1, n+1))
    ret[iu1] = utri
    ret.T[iu1] = utri
    np.fill_diagonal(ret, 1)
    return ret

def get_data(parc, twin='DZ'):
    '''
    Navigates through file tree and extracts test/retest FCs 
    '''
    # Yeo ordering
    master_dir = '../data/twins'
    tasks = ['rest', 'emotion', 'gambling', 'language', 'motor', 'relational', 'social', 'wm']
    FC, test, retest = {}, {}, {}
    for task in tasks:
        task_dir = master_dir + f'/{task.upper()}/origmat_{twin}_schaefer{parc}_tests.mat'
        f = h5py.File(task_dir, 'r')
        for k, v in f.items():
            test[task] = np.array(v)
        task_dir = master_dir + f'/{task.upper()}/origmat_{twin}_schaefer{parc}_retests.mat'
        f = h5py.File(task_dir, 'r')
        for k, v in f.items():
            retest[task] = np.array(v)
        FC[task] = np.concatenate((test[task], retest[task])) 
    return FC


def q1invm(q1, eig_thresh=0):
    U, S, V = scipy.linalg.svd(q1)
    s = np.diag(S)
    s[s < eig_thresh] = eig_thresh
    S = np.diag(s ** (-1 / 2))
    Q1_inv_sqrt = U * S * np.transpose(V)
    Q1_inv_sqrt = (Q1_inv_sqrt + np.transpose(Q1_inv_sqrt)) / 2
    return Q1_inv_sqrt


def qlog(q):
    U, S, V = scipy.linalg.svd(q)
    s = np.diag(S)
    S = np.diag(np.log(s))
    Q = U * S * np.transpose(V)
    return Q


def tangential(all_FC, ref):
    # Regularization for riemann
    if ref in ['riemann', 'kullback_sym', 'logeuclid']: 
        print("Adding regularization!")
        eye_mat = np.eye(all_FC.shape[1])
        scaling_mat = np.repeat(eye_mat[None, ...], all_FC.shape[0], axis=0)
        all_FC += scaling_mat
    u, s, vh = np.linalg.svd(all_FC[0], full_matrices=True)
    Cg = mean_covariance(all_FC, metric=ref)
    Q1_inv_sqrt = q1invm(Cg)
    Q = Q1_inv_sqrt @ all_FC @ Q1_inv_sqrt
    tangent_FC = np.array([qlog(a) for a in Q])
    return tangent_FC


def pca_recon(FC, pctComp=None):
    '''
    Reconstructs FC based on number of principle components
    '''
    if pctComp is None:
        return FC
    nRegions = FC.shape[1]
    FC = np.reshape(FC, (FC.shape[0], -1))
    nComp = int(FC.shape[0] * pctComp)
    mu = np.mean(FC, axis=0)
    pca_rest = sklearn.decomposition.PCA()
    pca_rest.fit(FC)
    SCORES = pca_rest.transform(FC)[:, :nComp]
    COEFFS = pca_rest.components_[:nComp, :]
    FC_recon = np.dot(SCORES, COEFFS)
    del SCORES, COEFFS
    FC_recon += mu
    FC_recon = np.reshape(FC_recon, (FC.shape[0], nRegions, nRegions))
    return FC_recon

def utri2mat(utri):
    n = int(-1 + np.sqrt(1 + 8 * len(utri))) // 2
    iu1 = np.tril_indices(n+1,-1)
    ret = np.empty((n+1, n+1))
    ret[iu1] = utri
    ret.T[iu1] = utri
    np.fill_diagonal(ret, 1)
    return ret

## Twin Subject ID Matching

In [25]:
FCs = get_data(100, twin=twin)
labels = np.tile(np.repeat(np.arange(0,FCs['rest'].shape[0]/4),2),2)
labels = labels.astype(int)
train_idx = np.arange(0,FCs['rest'].shape[0],2)
train_idx = train_idx.astype(int)
test_idx = np.arange(1, int(FCs['rest'].shape[0]), 2)
test_idx = test_idx.astype(int)
train_labels = labels[train_idx]
test_labels = labels[test_idx]

### KNN Approach - Manual Tangent Space, all Parcellations, all Tasks

In [10]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
accuracies = {}
lengths = {100:6441, 200:22791, 300:49141, 400:85491, 500:131841}
for parc in np.arange(100, 500, 100):
    common_FCs = get_data(parc, twin=twin)
    for task in ['rest', 'emotion', 'gambling', 'language', 'motor', 'relational', 'social', 'wm']:
        print(f'Analyzing {task}...')
        task_FCs = common_FCs[task]
        # Do optional transformations
        for ref in ['original']:
            # Start with a fresh batch of FCs
            FC = np.zeros((task_FCs.shape[0], parc+14, parc+14))
            for idx, utri in enumerate(task_FCs):
                FC[idx] = utri2mat(utri)
            # Do optional transformations
            if ref != 'original' and ref != 'pca':
                FC = tangential(FC, ref)
            elif ref == 'pca':
                print('Reconstructing with PCA')
                FC = pca_recon(FC, 0.3)
            else:
                pass
            # Convert back into flattened utriu vectors
            vec_FCs = np.zeros(
                (nFCs, lengths[parc]), dtype=np.float32)
            for idx, mat in enumerate(FC):
                vec_FCs[idx] = mat[np.triu_indices(mat.shape[0], k=1)]
            # Split into train and test sets
            train_FCs = vec_FCs[train_idx]
            test_FCs = vec_FCs[test_idx]
            # KNN Classifier
            neigh = KNeighborsClassifier(n_neighbors=1, metric='correlation')
            neigh.fit(train_FCs, train_labels)
            predicted = neigh.predict(test_FCs)
            acc1 = accuracy_score(test_labels, predicted)
            neigh.fit(test_FCs, test_labels)
            predicted = neigh.predict(train_FCs)
            acc2 = accuracy_score(test_labels, predicted)
            print(f'Accuracies: {acc1:.5f} and {acc2:.5f}')
            accuracies[f"{parc}:{task}:{ref}"] = (acc1 + acc2) / 2

Analyzing rest...
Accuracies: 0.32075 and 0.36792
Analyzing emotion...
Accuracies: 0.09434 and 0.09434
Analyzing gambling...
Accuracies: 0.15094 and 0.18868
Analyzing language...
Accuracies: 0.29245 and 0.33019
Analyzing motor...
Accuracies: 0.07547 and 0.09434
Analyzing relational...
Accuracies: 0.08491 and 0.07547
Analyzing social...
Accuracies: 0.20755 and 0.14151
Analyzing wm...
Accuracies: 0.25472 and 0.21698
Analyzing rest...
Accuracies: 0.40566 and 0.39623
Analyzing emotion...
Accuracies: 0.10377 and 0.09434
Analyzing gambling...


KeyboardInterrupt: 

### Save common tangent projected FCs

In [24]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
accuracies = {}
lengths = {100:6441, 200:22791, 300:49141, 400:85491, 500:131841}
for parc in np.arange(100, 500, 100):
    common_FCs = get_data(test_twin_ind, retest_twin_ind, parc, twin=twin)
    for task in ['rest', 'emotion', 'gambling', 'language', 'motor', 'relational', 'social', 'wm']:
        print(f'Analyzing {task}...')
        task_FCs = common_FCs[task]
        # Do optional transformations
        for ref in ['euclid', 'harmonic', 'kullback_sym', 'logeuclid', 'riemann']:
            # Start with a fresh batch of FCs
            FC = np.zeros((task_FCs.shape[0], parc+14, parc+14))
            for idx, utri in enumerate(task_FCs):
                FC[idx] = utri2mat(utri)
            # Do optional transformations
            FC = tangential(FC, ref)
            with open(f'../data/tangent_fcs/twins/common/{task}/{twin}_{parc}_{ref}.pickle', 'wb') as f:
                pickle.dump(FC, f)
            print(f"saved {ref}")
            

Analyzing rest...
saved euclid
saved harmonic
Adding regularization!
saved kullback_sym
Adding regularization!
saved logeuclid
Adding regularization!
saved riemann
Analyzing emotion...
saved euclid
saved harmonic
Adding regularization!
saved kullback_sym
Adding regularization!
saved logeuclid
Adding regularization!
saved riemann
Analyzing gambling...
saved euclid
saved harmonic
Adding regularization!
saved kullback_sym
Adding regularization!
saved logeuclid
Adding regularization!
saved riemann
Analyzing language...
saved euclid
saved harmonic
Adding regularization!
saved kullback_sym
Adding regularization!
saved logeuclid
Adding regularization!
saved riemann
Analyzing motor...
saved euclid
saved harmonic
Adding regularization!
saved kullback_sym
Adding regularization!
saved logeuclid
Adding regularization!
saved riemann
Analyzing relational...
saved euclid
saved harmonic
Adding regularization!
saved kullback_sym
Adding regularization!
saved logeuclid
Adding regularization!
saved rieman

In [12]:
accuracies

{'100:rest:original': 0.3443396226415094,
 '100:emotion:original': 0.09433962264150944,
 '100:gambling:original': 0.169811320754717,
 '100:language:original': 0.3113207547169811,
 '100:motor:original': 0.0849056603773585,
 '100:relational:original': 0.08018867924528301,
 '100:social:original': 0.17452830188679247,
 '100:wm:original': 0.2358490566037736,
 '200:rest:original': 0.4009433962264151,
 '200:emotion:original': 0.09905660377358491,
 '200:gambling:original': 0.2169811320754717,
 '200:language:original': 0.4056603773584906,
 '200:motor:original': 0.1320754716981132,
 '200:relational:original': 0.14622641509433962,
 '200:social:original': 0.25471698113207547,
 '200:wm:original': 0.3160377358490566,
 '300:rest:original': 0.4339622641509434,
 '300:emotion:original': 0.10377358490566038,
 '300:gambling:original': 0.25943396226415094,
 '300:language:original': 0.42924528301886794,
 '300:motor:original': 0.12735849056603774,
 '300:relational:original': 0.17452830188679247,
 '300:social

In [20]:
import csv
a_file = open(f"../results/twins/{twin}_twin_parcellations_new.csv", "w")

writer = csv.writer(a_file)
for key, value in accuracies.items():
    writer.writerow([key, value])
    
a_file.close()

### Tangent Space FCs - load premade tangent FCs, all parcellations, all tasks

In [29]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
accuracies = {}
twin = 'MZ'
lengths = {100:6441, 200:22791, 300:49141, 400:85491, 500:131841}
for parc in np.arange(100,500,100):
    print(f'Using {parc} region parcellation...')
    for task in ['rest', 'emotion', 'gambling', 'language', 'motor', 'relational', 'social', 'wm']:
        print(f'Analyzing {task}...')
        for ref in ['logeuclid']:
            # Start with a fresh batch of FCs
            with open(f'../data/tangent_fcs/twins/{task}/{twin}_{parc}_{ref}.pickle', 'rb') as f:
                FC = pickle.load(f)
            # Convert back into flattened utriu vectors
            vec_FCs = np.zeros((nFCs, lengths[parc]), dtype=np.float32)
            for idx, mat in enumerate(FC):
                vec_FCs[idx] = mat[np.triu_indices(mat.shape[0], k=1)]
            # Split into train and test sets
            train_FCs = vec_FCs[train_idx]
            test_FCs = vec_FCs[test_idx]
            # KNN Classifier
            neigh = KNeighborsClassifier(n_neighbors=1, metric='correlation')
            neigh.fit(train_FCs, train_labels)
            predicted = neigh.predict(test_FCs)
            acc = accuracy_score(test_labels, predicted)
            print(f'{acc:.5f} accuracy')
            accuracies[f"{parc}:{task}:{ref}"] = acc

Using 100 region parcellation...
Analyzing rest...
0.50000 accuracy
Analyzing emotion...
0.28879 accuracy
Analyzing gambling...
0.40086 accuracy
Analyzing language...
0.62069 accuracy
Analyzing motor...
0.43103 accuracy
Analyzing relational...
0.40948 accuracy
Analyzing social...
0.53017 accuracy
Analyzing wm...
0.56897 accuracy
Using 200 region parcellation...
Analyzing rest...
0.61638 accuracy
Analyzing emotion...
0.47845 accuracy
Analyzing gambling...
0.61207 accuracy
Analyzing language...
0.79741 accuracy
Analyzing motor...
0.68103 accuracy
Analyzing relational...
0.59483 accuracy
Analyzing social...
0.71121 accuracy
Analyzing wm...
0.71121 accuracy
Using 300 region parcellation...
Analyzing rest...
0.66810 accuracy
Analyzing emotion...
0.62069 accuracy
Analyzing gambling...
0.68534 accuracy
Analyzing language...
0.82759 accuracy
Analyzing motor...
0.78448 accuracy
Analyzing relational...
0.69397 accuracy
Analyzing social...


KeyboardInterrupt: 

In [38]:
train_FCs[220:]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [64]:
import csv
a_file = open(f"../results/twins/{twin}_twin_logeuclid.csv", "w")

writer = csv.writer(a_file)
for key, value in accuracies.items():
    writer.writerow([key, value])
    
a_file.close()